In [2]:
seeds = list(range(1000, 1 * 2000, 1000))
print(seeds)


[1000]


In [3]:
import multiprocessing, time

def print_squares(number):
    for i in range(number):
        print("square of {0} is {1}".format(i , i*i))
        time.sleep(0.1)

def print_cubes(number):
    for j in range(number):
        print("cube of {0} is {1}".format(j, j*j*j))
        time.sleep(0.1)

# if __name__ == "__main__":
    

In [5]:
process_1 = multiprocessing.Process(target = print_squares, args = (10,))
process_2 = multiprocessing.Process(target = print_cubes, args = (10,))

process_1.start()
process_2.start()

process_1.join()
process_2.join()

square of 0 is 0
cube of 0 is 0
square of 1 is 1
cube of 1 is 1
square of 2 is 4
cube of 2 is 8
square of 3 is 9
cube of 3 is 27
square of 4 is 16
cube of 4 is 64
square of 5 is 25
cube of 5 is 125
square of 6 is 36
cube of 6 is 216
square of 7 is 49
cube of 7 is 343
square of 8 is 64
cube of 8 is 512
square of 9 is 81
cube of 9 is 729


In [10]:
detla = float('-inf')
if 1e-7 > detla:
    print("hi")

hi


In [22]:
import torch
a = torch.rand((5,50,768))
b = torch.rand((5,50,768))

print(a.shape)
c = torch.cosine_similarity(a,b, dim=0)
d = torch.cosine_similarity(a,b, dim=1)
e = torch.cosine_similarity(a,b, dim=-1)
a_ = a[:,0,:]
b_ = b[:,0,:]
f = torch.cosine_similarity(a_,b_, dim=1)
print(c.shape)
print(d.shape)
print(e.shape, (1 - e).shape)

torch.Size([5, 50, 768])
torch.Size([50, 768])
torch.Size([5, 768])
torch.Size([5, 50]) torch.Size([5, 50])


In [25]:
t = 10
print(t % 7, t - t % 7)

3 7


In [37]:
t = 8
x = t % 7
y = t // 7
print(x, y)
# print(t % 7, t - (t / 7 - 1))

1 1


In [64]:
import torch.nn.functional as F

x = torch.randn(5, 3, 224, 224)
kernel_h = 32
kernel_w = 32
step = 32
n_channels = 3
patches = x.unfold(2, kernel_h, step).unfold(3, kernel_w, step).reshape(-1, n_channels, 49, 32, 32)#.permute(2, 3, 0, 1, 4, 5).reshape(-1, n_channels, kernel_h, kernel_w)

print(patches.shape)

# patches = x.unfold(1, kc, dc).unfold(2, kh, dh).unfold(3, kw, dw)
# unfold_shape = patches.size()
# patches = patches.contiguous().view(-1, kc, kh, kw)
# print(patches.shape)

# # Reshape back
# patches_orig = patches.view(unfold_shape)
# output_c = unfold_shape[1] * unfold_shape[4]
# output_h = unfold_shape[2] * unfold_shape[5]
# output_w = unfold_shape[3] * unfold_shape[6]
# patches_orig = patches_orig.permute(0, 1, 4, 2, 5, 3, 6).contiguous()
# patches_orig = patches_orig.view(1, output_c, output_h, output_w)

# # Check for equality
# print((patches_orig == x[:, :output_c, :output_h, :output_w]).all())
# # > tensor(True)

torch.Size([5, 3, 49, 32, 32])


In [52]:
x = torch.randn(1, 284, 143, 143)
kc, kh, kw = 128, 128, 128  # kernel size
dc, dh, dw = 128, 128, 128  # stride
# Pad to multiples of 128
# x = F.pad(x, (x.size(2)%kw // 2, x.size(2)%kw // 2,
#               x.size(1)%kh // 2, x.size(1)%kh // 2,
#               x.size(0)%kc // 2, x.size(0)%kc // 2))


patches = x.unfold(1, kc, dc).unfold(2, kh, dh).unfold(3, kw, dw)
unfold_shape = patches.size()
patches = patches.contiguous().view(-1, kc, kh, kw)
print(patches.shape)

torch.Size([2, 128, 128, 128])


In [50]:
print(1 * 284 * 143 * 143)
print(2 * 128 * 128 * 128)

5807516
4194304


In [42]:
x = torch.rand((10,10)).cuda()
print(x.shape)
y = x.clone()
print(x.shape)

torch.Size([10, 10])
torch.Size([10, 10])


In [80]:
import torch
import torch.nn.functional as F
import logging


def test_gumble_softmax():
    logging.basicConfig(level="INFO")
    mask = torch.rand(5)
    k = 3
    mask = torch.tensor([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.9])
    mask.requires_grad = True
    optimizer = torch.optim.Adam([mask], lr=1e-2, betas=(0.9, 0.999))
    for step in range(10):
        optimizer.zero_grad()
        mask_new = torch.sigmoid(mask)
        mask_flipped = 1 - mask_new
        # print("new", mask_new)
        # print("flip", mask_flipped)
        v = torch.stack((torch.log(mask_new), torch.log(mask_flipped)), dim=-1)
        # print("v", v)
        hard_mask = torch.nn.functional.gumbel_softmax(v, 0.8, True)
        # print(hard_mask)
        mini_loss = torch.sum(hard_mask[:, 0])  # minimal sum
        k_loss = 0 * (torch.sum(hard_mask[:, 0]) - k) ** 2  # choose only k values
        loss = k_loss + mini_loss
        logging.info('epoch: {}, mini_loss: {}, kloss: {}'.format(step, mini_loss.item(), k_loss.item()))
        print(mask)
        loss.backward()
        optimizer.step()
    assert k - 20 <= mini_loss.item() <= k + 20
    print(hard_mask)
test_gumble_softmax()

INFO:root:epoch: 0, mini_loss: 3.0, kloss: 0.0
INFO:root:epoch: 1, mini_loss: 3.0, kloss: 0.0
INFO:root:epoch: 2, mini_loss: 4.0, kloss: 0.0
INFO:root:epoch: 3, mini_loss: 3.0, kloss: 0.0
INFO:root:epoch: 4, mini_loss: 3.0, kloss: 0.0
INFO:root:epoch: 5, mini_loss: 6.0, kloss: 0.0
INFO:root:epoch: 6, mini_loss: 6.0, kloss: 0.0
INFO:root:epoch: 7, mini_loss: 1.0, kloss: 0.0
INFO:root:epoch: 8, mini_loss: 5.0, kloss: 0.0
INFO:root:epoch: 9, mini_loss: 5.0, kloss: 0.0


tensor([0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.9000],
       requires_grad=True)
tensor([ 1.8626e-09, -9.3132e-10,  1.8626e-09,  9.3132e-10,  5.5879e-09,
         9.3132e-10,  8.9000e-01], requires_grad=True)
tensor([-0.0091, -0.0100, -0.0100, -0.0097, -0.0079, -0.0093,  0.8823],
       requires_grad=True)
tensor([-0.0187, -0.0185, -0.0182, -0.0185, -0.0167, -0.0169,  0.8739],
       requires_grad=True)
tensor([-0.0279, -0.0275, -0.0270, -0.0275, -0.0259, -0.0254,  0.8648],
       requires_grad=True)
tensor([-0.0370, -0.0365, -0.0363, -0.0369, -0.0353, -0.0344,  0.8555],
       requires_grad=True)
tensor([-0.0463, -0.0452, -0.0458, -0.0465, -0.0442, -0.0425,  0.8461],
       requires_grad=True)
tensor([-0.0559, -0.0540, -0.0549, -0.0550, -0.0534, -0.0497,  0.8374],
       requires_grad=True)
tensor([-0.0657, -0.0623, -0.0634, -0.0629, -0.0622, -0.0576,  0.8292],
       requires_grad=True)
tensor([-0.0756, -0.0709, -0.0723, -0.0714, -0.0712, -0.0660,  0.8204],
       requires

In [4]:
# 5, 50, 768
import torch
a = torch.ones((5, 50, 768))
a[:, 0, :] = 0
print(a[:, 0, :].sum())
print(a[:, 0, :].shape)
print(a[:, 1, :].sum())

tensor(0.)
torch.Size([5, 768])
tensor(3840.)
